# 833-JinxML - Predict Time

## Load Dataset

In [41]:
from tlol.datasets.replay_dataset import *
dataset = TLoLReplayDataset("./full_db")

## Dataset

In [64]:
tm_x_s = []
tm_y_s = []

for i in range(1): # len(dataset)
    cur  = dataset[i]
    tm_x = np.array([cur["raw"].iloc[0:2, 0]]) / 300.0
    tm_x = torch.Tensor(tm_x)

    tm_y = np.array([cur["raw"].iloc[0:2, 0]]) / 300.0
    tm_y = torch.Tensor(tm_y)
    
    tm_x_s.append(tm_x)
    tm_y_s.append(tm_y)

## Model

In [65]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np

seed = 1
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

class TimePredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 1)
    
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        return x

model = TimePredictor()

## Training

In [66]:
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.cuda.amp.grad_scaler import GradScaler

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = "cpu" # torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

amp = False
scaler = GradScaler()

n_epochs = 10_000

losses = []

for epoch in range(1, n_epochs + 1):
    for tm_x, tm_y in zip(tm_x_s, tm_y_s):
        optimizer.zero_grad()

        tm_x = tm_x.to(device)
        tm_y = tm_y.to(device)

        preds = model(tm_x)

        # print(preds, tm_x, tm_y)

        loss = criterion(preds, tm_y)
        losses.append(loss.detach().numpy())
        
        loss.backward()
        optimizer.step()

    if epoch % 100 == 0:
        print(epoch, loss.item(), tm_x * 300, tm_y * 300, preds * 300)

plt.plot(losses)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x2 and 1x1)